In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

from sklearn.model_selection import train_test_split

import pickle
import gzip
import time

Using TensorFlow backend.
/home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3263223534740639722, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3482191812518279862
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17576692117659392542
 physical_device_desc: "device: XLA_GPU device"]

In [5]:
np.random.seed(7)
%matplotlib inline

In [6]:
DIR_PATH = 'dataset/notracking/mnist/'

train_df = pd.read_csv(DIR_PATH + 'train.csv')
pred_df = pd.read_csv(DIR_PATH + 'test.csv')

train_df, test_df = train_test_split(train_df, test_size=0.2)

sub = pd.read_csv(DIR_PATH + 'sample_submission.csv')

In [7]:
def load_data():
    f = gzip.open('dataset/notracking/mnist.pkl.gz', 'rb')
    f.seek(0)
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

df = load_data()

X_train = np.concatenate((df[0][0], df[1][0], df[2][0]), axis=0)
y_train = np.concatenate((df[0][1], df[1][1], df[2][1]), axis=0)

temp = pd.DataFrame({'label' : y_train})

train_df = pd.concat([temp, pd.DataFrame(X_train)], axis=1)
train_df, test_df = train_test_split(train_df, test_size=0.15)

pred_df = pd.read_csv(DIR_PATH + 'test.csv')
sub = pd.read_csv(DIR_PATH + 'sample_submission.csv')

In [8]:
def plot(dataset, index):
    k = dataset[index]
    k = k.reshape((28, 28))
    #plt.title('Label is {}'.format(training_data[1][index]))
    plt.imshow(k, cmap='gray')

In [9]:
def one_hot(j):
    # input is the target dataset of shape (1, m) where m is the number of data points
    # returns a 2 dimensional array of shape (10, m) where each target value is converted to a one hot encoding
    # Look at the next block of code for a better understanding of one hot encoding
    n = j.shape[0]
    new_array = np.zeros((10, n))
    index = 0
    for res in j:
        new_array[res][index] = 1.0
        index = index + 1
    return new_array

In [10]:
X_train = train_df.loc[:, train_df.columns != 'label'].values
y_train = one_hot(train_df['label'].values).T

X_test = test_df.loc[:, test_df.columns != 'label'].values
y_test = one_hot(test_df['label'].values).T

X_pred = pred_df.values

In [11]:
# create model
nn_model = Sequential()
nn_model.add(Dense(50, input_dim=784, activation='relu'))
nn_model.add(Dropout(0.25))
nn_model.add(Dense(35, activation = 'relu'))
nn_model.add(Dense(21, activation = 'relu'))
nn_model.add(Dense(10, activation='softmax'))

W0823 21:10:52.588860 139864239109952 deprecation_wrapper.py:119] From /home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 21:10:52.590542 139864239109952 deprecation_wrapper.py:119] From /home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 21:10:52.592758 139864239109952 deprecation_wrapper.py:119] From /home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0823 21:10:52.604381 139864239109952 deprecation_wrapper.py:119] From /home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python

In [12]:
nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

W0823 21:10:52.643806 139864239109952 deprecation_wrapper.py:119] From /home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0823 21:10:52.659448 139864239109952 deprecation_wrapper.py:119] From /home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.



In [13]:
start_time = time.time()
nn_model.fit(X_train, y_train, epochs=150, batch_size=10)
end_time = time.time()
print(f"Time taken for NN Execution is {end_time - start_time}")

W0823 21:10:52.729249 139864239109952 deprecation.py:323] From /home/shashi/.local/share/virtualenvs/python-code-gqSjhHCu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/150
59500/59500 [==============================] - 5s 91us/step - loss: 0.3844 - categorical_accuracy: 0.8813
Epoch 2/150
59500/59500 [==============================] - 5s 86us/step - loss: 0.2260 - categorical_accuracy: 0.9308
Epoch 3/150
59500/59500 [==============================] - 5s 87us/step - loss: 0.1933 - categorical_accuracy: 0.9409
Epoch 4/150
59500/59500 [==============================] - 5s 86us/step - loss: 0.1777 - categorical_accuracy: 0.9450
Epoch 5/150
59500/59500 [==============================] - 5s 86us/step - loss: 0.1634 - categorical_accuracy: 0.9501
Epoch 6/150
59500/59500 [==============================] - 5s 86us/step - loss: 0.1541 - categorical_accuracy: 0.9521
Epoch 7/150
59500/59500 [==============================] - 5s 87us/step - loss: 0.1478 - categorical_accuracy: 0.9533
Epoch 8/150
59500/59500 [==============================] - 5s 92us/step - loss: 0.1414 - categorical_accuracy: 0.9561
Epoch 9/150
59500/59500 [==============================]

59500/59500 [==============================] - 5s 89us/step - loss: 0.0800 - categorical_accuracy: 0.9747
Epoch 70/150
59500/59500 [==============================] - 5s 89us/step - loss: 0.0784 - categorical_accuracy: 0.9758
Epoch 71/150
59500/59500 [==============================] - 5s 91us/step - loss: 0.0775 - categorical_accuracy: 0.9756
Epoch 72/150
59500/59500 [==============================] - 5s 88us/step - loss: 0.0755 - categorical_accuracy: 0.9764
Epoch 73/150
59500/59500 [==============================] - 5s 89us/step - loss: 0.0779 - categorical_accuracy: 0.9763
Epoch 74/150
59500/59500 [==============================] - 5s 88us/step - loss: 0.0776 - categorical_accuracy: 0.9751
Epoch 75/150
59500/59500 [==============================] - 5s 89us/step - loss: 0.0782 - categorical_accuracy: 0.9750
Epoch 76/150
59500/59500 [==============================] - 5s 88us/step - loss: 0.0750 - categorical_accuracy: 0.9764
Epoch 77/150
59500/59500 [==============================] - 5

59500/59500 [==============================] - 5s 91us/step - loss: 0.0686 - categorical_accuracy: 0.9787
Epoch 138/150
59500/59500 [==============================] - 6s 94us/step - loss: 0.0645 - categorical_accuracy: 0.9798
Epoch 139/150
59500/59500 [==============================] - 6s 96us/step - loss: 0.0683 - categorical_accuracy: 0.9792
Epoch 140/150
59500/59500 [==============================] - 6s 100us/step - loss: 0.0657 - categorical_accuracy: 0.9796
Epoch 141/150
59500/59500 [==============================] - 6s 101us/step - loss: 0.0674 - categorical_accuracy: 0.9787
Epoch 142/150
59500/59500 [==============================] - 6s 99us/step - loss: 0.0685 - categorical_accuracy: 0.9796
Epoch 143/150
59500/59500 [==============================] - 6s 94us/step - loss: 0.0664 - categorical_accuracy: 0.9795
Epoch 144/150
59500/59500 [==============================] - 6s 93us/step - loss: 0.0682 - categorical_accuracy: 0.9790
Epoch 145/150
59500/59500 [=========================

Accuracy on Training Data

In [14]:
scores_train = nn_model.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (nn_model.metrics_names[1], scores_train[1]*100))

59500/59500 [==============================] - 1s 12us/step

categorical_accuracy: 99.58%


Accuracy on Test Data

In [15]:
scores_test = nn_model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (nn_model.metrics_names[1], scores_test[1]*100))

10500/10500 [==============================] - 0s 11us/step

categorical_accuracy: 97.13%


In [16]:
predictions = nn_model.predict(X_pred)
predictions = np.argmax(predictions, axis = 1)
predictions.shape

(28000,)

In [17]:
#plot(X_test, 2)

In [18]:
sub['Label'] = predictions
sub.to_csv(DIR_PATH + "submission_1.csv", index = False)